<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import google earth engine API**

In [1]:
import ee

**Authenticate google earth engine and your intialize the cloud project**

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

**Import libraries needed to run the code**

In [3]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
import gdown
import json
from IPython.display import Image, display
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 34.9 MB/s eta 0:00:00
Mounted at /content/drive


**Visualize Sentinel-2 Scene**

In [ ]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

**Compute vegetation indices**

In [ ]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('B3'),
    'RED': image.select('B4'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('B3'),
    'GREEN': image.select('B4'),
    'BLUE': image.select('B2'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
    'BLUE': image.select('B2'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])
# Print image bands
print(image_final.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

**Load training data from google drive to google colab**

In [6]:
# Specify the Google Drive file ID of the shapefile
file_id = '1iA9L6tn0dwVlABPngFAgjRxd8xXET1c4'
output_path = '/content/drive/My Drive/Colab Notebooks/Data/TrainingSet_SabieCroc/Training_setSc.shp'
url = f'https://drive.google.com/uc?id={file_id}'

# Read the shapefile into a GeoDataFrame
training_data = gpd.read_file(output_path)

#print (training_data)

# Convert the GeoDataFrame to GeoJSON and save it
training_data.to_file('training_set.geojson', driver='GeoJSON')

# Convert the GeoDataFrame to a Feature Collection (assuming you want to use Earth Engine)
training_subset = ee.FeatureCollection('training_set.geojson')
print(training_subset)

# Print the contents of the GeoJSON file
with open('training_set.geojson', 'r') as f:
    #geojson_data = json.load(f)

print(json.dumps(geojson_data, indent=2))


IndentationError: expected an indented block after 'with' statement on line 19 (<ipython-input-6-a5571c98177c>, line 22)

**Sample regions from the image at training data locations**

In [ ]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI',];

# Sample regions from the image at training feature locations
training_dataset = image_final.select(bands).sampleRegions(
    collection=training_subset,
    properties=['ID', 'LULC_Class'],
    scale=10
)

**Run the classification using random forest**

In [ ]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_subset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

**Defining a colour palette for the classified map and viweing the map**

In [ ]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m




EEException: Collection.loadTable: Collection asset 'training_set.geojson' not found.